# **EN Word2Vec**

〽️ Using xml file : needs preprocessing such as deleting tags (/content) 






In [6]:
import re
from lxml import etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
# nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

**(1) Loading Data with url**

In [3]:
#download dataset by url 
urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip") 


with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:

  target_text = etree.parse(z.open('ted_en-20160408.xml', 'r'))

  parse_text = '\n'.join(target_text.xpath('//content/text()'))  #save only words inside content tag 

parse_text[:300]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit"

**(2) Preprocessing Data**

In [7]:

#1. delete background audio, laughter.... which has parenthesis 
content_text = re.sub(r'\([^)]*\)', '', parse_text)

#tokenize the sentences  
sent_text=sent_tokenize(content_text)


#delete non word, non number info, and make words to lower 
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower()) 

     normalized_text.append(tokens)
     
#tokenize the words 
result = [word_tokenize(sentence) for sentence in normalized_text]

for line in result[:10]: 
    print(line) #visualize the result data :: how data looks like 
    #seperated with sentences -> tokenized words  

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']
['consider', 'facit']
['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them']
['facit', 'was', 'a', 'fantastic', 'company']
['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world']
['everybody', 'used', 'them']
['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along']
['they', 'continued', 'doing', 'exactly', 'the', 'same']


**(3) Training the dataset with Word2Vec**

In [9]:
from gensim.models import Word2Vec

#use word2vec to train the dataset 

model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)
#sentences = tokenized word sentence 
#size = embedding vector size 
#windows = how many words we are going to see front, back 
#min_count = if under 5, lose the word 
#workers= processer num
#sd = 0 : CBOW 
#sd =1 : skip gram


model_result = model.wv.most_similar("woman")
print(model_result)

[('girl', 0.8535995483398438), ('man', 0.8482182025909424), ('boy', 0.805486798286438), ('lady', 0.7835748195648193), ('child', 0.7515976428985596), ('kid', 0.7249900102615356), ('nurse', 0.7235825657844543), ('soldier', 0.7031956911087036), ('guy', 0.700143575668335), ('gentleman', 0.6947692632675171)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


**(4) Saving the Model**

In [11]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('eng_w2v') # save by eng_w2v name 

loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")  #load by using this function 

#can print out same result as above 
model_result = model.wv.most_similar("woman")
print(model_result)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[('girl', 0.8535995483398438), ('man', 0.8482182025909424), ('boy', 0.805486798286438), ('lady', 0.7835748195648193), ('child', 0.7515976428985596), ('kid', 0.7249900102615356), ('nurse', 0.7235825657844543), ('soldier', 0.7031956911087036), ('guy', 0.700143575668335), ('gentleman', 0.6947692632675171)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# **KOREAN Word2Vec**

〽️ Using naver movie reviews




In [0]:
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

# !pip install konlpy

**(1) Loading Data with url**

In [16]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
train_data = pd.read_table('ratings.txt')
train_data = train_data.dropna(how = 'any') # delete null value row 
train_data[:10] 

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


**(2) Preprocessing Data**

In [0]:
#delete other words than korean 
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] #deleting stop words

okt = Okt() #korean word tokenizer 
tokenized_data = []


for sentence in train_data['document']:

    temp_X = okt.morphs(sentence, stem=True) # tokenize
    temp_X = [word for word in temp_X if not word in stopwords] # deleting stop words 
    tokenized_data.append(temp_X)

**(3) Training the dataset with Word2Vec**

In [18]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 1) #use skip-gram this time 
model.wv.vectors.shape 
#the size was 100, so embedded matrix has shape of (16477, 100)


(16477, 100)

In [19]:
#look through model 

print(model.wv.most_similar("김수현"))

[('김은숙', 0.8348084688186646), ('노희경', 0.8332891464233398), ('강지환', 0.8175852298736572), ('류덕환', 0.8153785467147827), ('이민호', 0.8136894106864929), ('차인표', 0.8123669624328613), ('최수종', 0.810583770275116), ('한석규', 0.8068250417709351), ('찰떡', 0.8067680597305298), ('이상우', 0.8042839765548706)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# **TRAINING word2vec**

〽️ two ways of using training : 

1) train own data by word embedding 

2) use pretrained embedding vector









**Train by embedding layer vectors**

(1) tokenize dataset 

In [21]:
#embedding layer 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1] #sentimental analysis 1-pos, 0-neg 

#1 tokenize each sentence 
t = Tokenizer()
t.fit_on_texts(sentences)


#get the total vocab size 
vocab_size = len(t.word_index) + 1

print(vocab_size)



16


(2) integer encoding : make them into nums 

In [22]:

#int encoding (make the words into integers)
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)


#get the max len :: for padding 
max_len=max(len(l) for l in X_encoded)
print(max_len)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]
4


(3) pad according to longest sentence length which is 4 (all to 0)

In [23]:
#padd as the max len 

X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


(4) train dataset using embedding layer

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len)) # 모든 임베딩 벡터는 4차원.

#EMBEDDING LAYER needs......

#vocab_size : length of total word vocab (vocabulary amount)
#output_dim : embedding vector dimension 
#input_length: input sequence ; max length of words in one sentence (every sentence needs to be padded )


model.add(Flatten()) # to put in dense layer flatten to 1-dimension 
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

#train dataset 
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.6986 - acc: 0.5714
Epoch 2/100
1/1 - 0s - loss: 0.6972 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6959 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6946 - acc: 0.5714
Epoch 5/100
1/1 - 0s - loss: 0.6933 - acc: 0.5714
Epoch 6/100
1/1 - 0s - loss: 0.6921 - acc: 0.5714
Epoch 7/100
1/1 - 0s - loss: 0.6908 - acc: 0.5714
Epoch 8/100
1/1 - 0s - loss: 0.6895 - acc: 0.5714
Epoch 9/100
1/1 - 0s - loss: 0.6882 - acc: 0.5714
Epoch 10/100
1/1 - 0s - loss: 0.6869 - acc: 0.5714
Epoch 11/100
1/1 - 0s - loss: 0.6856 - acc: 0.5714
Epoch 12/100
1/1 - 0s - loss: 0.6843 - acc: 0.5714
Epoch 13/100
1/1 - 0s - loss: 0.6831 - acc: 0.5714
Epoch 14/100
1/1 - 0s - loss: 0.6818 - acc: 0.5714
Epoch 15/100
1/1 - 0s - loss: 0.6805 - acc: 0.5714
Epoch 16/100
1/1 - 0s - loss: 0.6792 - acc: 0.5714
Epoch 17/100
1/1 - 0s - loss: 0.6779 - acc: 0.7143
Epoch 18/100
1/1 - 0s - loss: 0.6766 - acc: 0.7143
Epoch 19/100
1/1 - 0s - loss: 0.6753 - acc: 0.7143
Epoch 20/100
1/1 - 0s - loss: 0.6740 - a

**USING pre-trained wordvectors**

〽️ example of English, Korean version 






**Load pretrained wordvector (1)ENGLISH (2)KOREAN**

In [44]:
#english version -- too big 
import gensim

#use google's pretrained wordvector 
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  
print(word2vec_model.vectors.shape) #(3000000, 300)


--2020-06-01 10:17:24--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.93.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.93.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  90.3MB/s    in 17s     

2020-06-01 10:17:42 (91.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [45]:
embedding_matrix = np.zeros((vocab_size, 300))#make matrix according to the size of pretrained vector matrix 
#since the shape has 300 columns , make a zero matrix with this amount 
np.shape(embedding_matrix)

(16, 300)

In [48]:
#function for checking if the word is in the vector matrix 

def get_vector(word):
    if word in word2vec_model:

        return word2vec_model[word]
    else:

        return None

#use function to get the word vector matrix for our words 
for word, i in t.word_index.items(): 
    temp = get_vector(word) 
    if temp is not None: 
        embedding_matrix[i] = temp 

print(word2vec_model['great']) 
print(len(word2vec_model['great']) ) #has 300 columns 

[ 7.17773438e-02  2.08007812e-01 -2.84423828e-02  1.78710938e-01
  1.32812500e-01 -9.96093750e-02  9.61914062e-02 -1.16699219e-01
 -8.54492188e-03  1.48437500e-01 -3.34472656e-02 -1.85546875e-01
  4.10156250e-02 -8.98437500e-02  2.17285156e-02  6.93359375e-02
  1.80664062e-01  2.22656250e-01 -1.00585938e-01 -6.93359375e-02
  1.04427338e-04  1.60156250e-01  4.07714844e-02  7.37304688e-02
  1.53320312e-01  6.78710938e-02 -1.03027344e-01  4.17480469e-02
  4.27246094e-02 -1.10351562e-01 -6.68945312e-02  4.19921875e-02
  2.50000000e-01  2.12890625e-01  1.59179688e-01  1.44653320e-02
 -4.88281250e-02  1.39770508e-02  3.55529785e-03  2.09960938e-01
  1.52343750e-01 -7.32421875e-02  2.16796875e-01 -5.76171875e-02
 -2.84423828e-02 -3.60107422e-03  1.52343750e-01 -2.63671875e-02
  2.13623047e-02 -1.51367188e-01  1.04003906e-01  3.18359375e-01
 -1.85546875e-01  3.68652344e-02 -1.10839844e-01 -3.17382812e-02
 -1.01562500e-01 -1.21093750e-01  3.22265625e-01 -7.32421875e-02
 -1.52343750e-01  2.67578

In [49]:
#train model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.6410 - acc: 0.8571
Epoch 2/100
1/1 - 0s - loss: 0.6235 - acc: 1.0000
Epoch 3/100
1/1 - 0s - loss: 0.6065 - acc: 1.0000
Epoch 4/100
1/1 - 0s - loss: 0.5901 - acc: 1.0000
Epoch 5/100
1/1 - 0s - loss: 0.5742 - acc: 1.0000
Epoch 6/100
1/1 - 0s - loss: 0.5588 - acc: 1.0000
Epoch 7/100
1/1 - 0s - loss: 0.5439 - acc: 1.0000
Epoch 8/100
1/1 - 0s - loss: 0.5296 - acc: 1.0000
Epoch 9/100
1/1 - 0s - loss: 0.5158 - acc: 1.0000
Epoch 10/100
1/1 - 0s - loss: 0.5024 - acc: 1.0000
Epoch 11/100
1/1 - 0s - loss: 0.4895 - acc: 1.0000
Epoch 12/100
1/1 - 0s - loss: 0.4771 - acc: 1.0000
Epoch 13/100
1/1 - 0s - loss: 0.4651 - acc: 1.0000
Epoch 14/100
1/1 - 0s - loss: 0.4535 - acc: 1.0000
Epoch 15/100
1/1 - 0s - loss: 0.4423 - acc: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.4315 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.4210 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4109 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4012 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.3918 - a

**Using GloVe**

In [34]:
#using glove.6B.100d for pretrained embedding 

import numpy as np
embedding_dict = dict()
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()

len(embedding_dict) #has 400000 words 

print(embedding_dict['respectable']) #look at the embedding for respectable , has 100 length (column) , row is word 

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [35]:
embedding_matrix = np.zeros((vocab_size, 100))
 #make a matrix filled with zeros -> fill with the pretrained embedding matrix 
 #100 since the pretrained datset has 100 dimension 

np.shape(embedding_matrix)

(16, 100)

In [38]:
print(t.word_index.items()) #tokenizer used above 


dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [42]:

for word, i in t.word_index.items():  #get key, value from dictionary 
    temp = embedding_dict.get(word)  #get the embedding values for each word from pretrained matrix 

    if temp is not None: 
        embedding_matrix[i] = temp #if the word is inside the pretrained matrix -- use the value and implement to embedding matrix 



embedding_matrix[:1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [43]:

#create model 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()

e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False) #the weights are now the pretrained model 

model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)




Epoch 1/100
1/1 - 0s - loss: 0.8681 - acc: 0.5714
Epoch 2/100
1/1 - 0s - loss: 0.8408 - acc: 0.4286
Epoch 3/100
1/1 - 0s - loss: 0.8144 - acc: 0.4286
Epoch 4/100
1/1 - 0s - loss: 0.7888 - acc: 0.4286
Epoch 5/100
1/1 - 0s - loss: 0.7641 - acc: 0.4286
Epoch 6/100
1/1 - 0s - loss: 0.7403 - acc: 0.4286
Epoch 7/100
1/1 - 0s - loss: 0.7175 - acc: 0.4286
Epoch 8/100
1/1 - 0s - loss: 0.6957 - acc: 0.4286
Epoch 9/100
1/1 - 0s - loss: 0.6749 - acc: 0.4286
Epoch 10/100
1/1 - 0s - loss: 0.6551 - acc: 0.4286
Epoch 11/100
1/1 - 0s - loss: 0.6362 - acc: 0.5714
Epoch 12/100
1/1 - 0s - loss: 0.6184 - acc: 0.5714
Epoch 13/100
1/1 - 0s - loss: 0.6014 - acc: 0.7143
Epoch 14/100
1/1 - 0s - loss: 0.5854 - acc: 0.7143
Epoch 15/100
1/1 - 0s - loss: 0.5702 - acc: 0.7143
Epoch 16/100
1/1 - 0s - loss: 0.5558 - acc: 0.7143
Epoch 17/100
1/1 - 0s - loss: 0.5422 - acc: 0.7143
Epoch 18/100
1/1 - 0s - loss: 0.5293 - acc: 0.7143
Epoch 19/100
1/1 - 0s - loss: 0.5171 - acc: 0.7143
Epoch 20/100
1/1 - 0s - loss: 0.5054 - a